# 01. Hello RAG (Simple In-Memory)
Welcome to your first RAG Lab! In this notebook, we will:
1. Connect to Llama3 using `langchain-ollama`.
2. Perform a simple generation.
3. Build a RAG system using `langchain-qdrant`.

In [ ]:
import os
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# Initialize Llama3
llm = ChatOllama(
    base_url=os.environ.get('OLLAMA_BASE_URL'),
    model='llama3'
)
print('LLM Initialized')

In [ ]:
# 1. Simple Generation Test
response = llm.invoke('Tell me a one-sentence joke about Docker.')
print(response.content)

## Simple RAG using Qdrant
We will use `QdrantVectorStore` connected to our Qdrant container.

In [8]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_qdrant import QdrantVectorStore
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.documents import Document

# Initialize Embeddings (running locally on CPU)
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
print('Embeddings model loaded')

Embeddings model loaded


In [9]:
# Create some dummy data
text_data = [
    'SnapLogic is a powerful integration platform.',
    'Docker helps you containerize applications.',
    'RAG stands for Retrieval-Augmented Generation.',
    'Llama3 is a large language model by Meta.'
]
docs = [Document(page_content=t) for t in text_data]

In [11]:
# Create a vector store
url = os.environ.get('QDRANT_URL')
qdrant = QdrantVectorStore.from_documents(
    docs,
    embeddings,
    url=url,
    prefer_grpc=False,
    collection_name='hello_rag'
)
print('Documents indexed in Qdrant!')

Documents indexed in Qdrant!


In [12]:
# Perform a Search
query = 'What do you think RAG is?'
found_docs = qdrant.similarity_search(query, k=1)
print(f'Top result: {found_docs[0].page_content}')

Top result: RAG stands for Retrievel Agumented Generation


In [13]:
# Connect it to the LLM (The RAG part)
from langchain_core.runnables import RunnablePassthrough

retriever = qdrant.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print(chain.invoke('What does RAG stand for?'))

According to the provided context, RAG stands for:

* Retrievel Agumented Generation (based on the first document)
* Retrieval-Augmented Generation (based on the other three documents)
